#📌 Extracão

In [8]:
import pandas as pd
import requests
import json


url_dados = 'https://raw.githubusercontent.com/jhonatanwsds/TelecomX_BR/refs/heads/main/TelecomX_Data.json'

try:
    response = requests.get(url_dados)
    response.raise_for_status()
    data = json.loads(response.text)

    dados_clientes = pd.json_normalize(data)

    print("Dados extraídos e organizados com sucesso da forma simples!")
    display(dados_clientes.head())

except Exception as e:
    print(f"Ocorreu um erro: {e}")

Dados extraídos e organizados com sucesso da forma simples!


,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


#🔧 Transformação

In [13]:
import pandas as pd
import numpy as np

print("--- Iniciando a Transformação e Limpeza dos Dados ---")

# --- 1. CORREÇÃO DOS DADOS ---

dados_clientes['account.Charges.Total'] = pd.to_numeric(dados_clientes['account.Charges.Total'], errors='coerce')

dados_clientes['Churn'] = dados_clientes['Churn'].replace('', np.nan)

dados_clientes.dropna(inplace=True)

print("\n[PASSO 1] Dados problemáticos corrigidos e removidos.")

# --- 2. VERIFICAÇÃO FINAL ---

print("\n--- Verificação Pós-Limpeza ---")

# Confirma que não há mais valores ausentes
print(f"\nValores ausentes restantes: {dados_clientes.isnull().sum().sum()}")

# Confirma que não há clientes duplicados
print(f"Clientes duplicados restantes: {dados_clientes['customerID'].duplicated().sum()}")

# Mostra os valores únicos na coluna Churn para provar que o '' sumiu
print(f"Valores únicos em 'Churn' agora: {pd.unique(dados_clientes['Churn'])}")

# Mostra o status final do DataFrame
print("\n--- Estrutura Final dos Dados (Pronto para Análise) ---")
dados_clientes.info()

--- Iniciando a Transformação e Limpeza dos Dados ---

[PASSO 1] Dados problemáticos corrigidos e removidos.

--- Verificação Pós-Limpeza ---

Valores ausentes restantes: 0
Clientes duplicados restantes: 0
Valores únicos em 'Churn' agora: ['No' 'Yes']

--- Estrutura Final dos Dados (Pronto para Análise) ---
<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7032 non-null   object 
 1   Churn                      7032 non-null   object 
 2   customer.gender            7032 non-null   object 
 3   customer.SeniorCitizen     7032 non-null   int64  
 4   customer.Partner           7032 non-null   object 
 5   customer.Dependents        7032 non-null   object 
 6   customer.tenure            7032 non-null   int64  
 7   phone.PhoneService         7032 non-null   object 
 8   phone.MultipleLines     

Criando a coluna 'Contas_Diarias'

In [14]:
import pandas as pd

print("--- Criando a coluna 'Contas_Diarias' ---")

dados_clientes['Contas_Diarias'] = dados_clientes['account.Charges.Monthly'] / 30

print("\nColuna 'Contas_Diarias' criada com sucesso. Veja a amostra:")
print(dados_clientes[['customerID', 'account.Charges.Monthly', 'Contas_Diarias']].head())


print("\n--- Estrutura Final dos Dados com a Nova Coluna ---")
dados_clientes.info()

--- Criando a coluna 'Contas_Diarias' ---

Coluna 'Contas_Diarias' criada com sucesso. Veja a amostra:
   customerID  account.Charges.Monthly  Contas_Diarias
0  0002-ORFBO                     65.6        2.186667
1  0003-MKNFE                     59.9        1.996667
2  0004-TLHLJ                     73.9        2.463333
3  0011-IGKFF                     98.0        3.266667
4  0013-EXCHZ                     83.9        2.796667

--- Estrutura Final dos Dados com a Nova Coluna ---
<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7266
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7032 non-null   object 
 1   Churn                      7032 non-null   object 
 2   customer.gender            7032 non-null   object 
 3   customer.SeniorCitizen     7032 non-null   int64  
 4   customer.Partner           7032 non-null   object 
 5   customer.Depend

Padronização

In [15]:
import pandas as pd

print("--- Padronizando e Transformando os Dados ---")

# --- 1. Renomeando as colunas para nomes mais simples ---
print("\nNomes das colunas ANTES da padronização:")
print(dados_clientes.columns.tolist())

dados_clientes.columns = [col.split('.')[-1] for col in dados_clientes.columns]

print("\nNomes das colunas DEPOIS da padronização:")
print(dados_clientes.columns.tolist())


# --- 2. Convertendo colunas 'Yes'/'No' para Binário (1/0) ---
colunas_binarias = [
    'Churn',
    'Partner',
    'Dependents',
    'PhoneService',
    'PaperlessBilling'
]

mapa_sim_nao = {'Yes': 1, 'No': 0}

for coluna in colunas_binarias:
    dados_clientes[coluna] = dados_clientes[coluna].replace(mapa_sim_nao)

print(f"\nColunas {colunas_binarias} convertidas para 1 e 0.")
print("\nAmostra dos dados após a conversão:")
print(dados_clientes[colunas_binarias].head())


# --- Verificação Final ---
print("\n--- Estrutura Final dos Dados Após Padronização ---")
dados_clientes.info()

--- Padronizando e Transformando os Dados ---

Nomes das colunas ANTES da padronização:
['customerID', 'Churn', 'customer.gender', 'customer.SeniorCitizen', 'customer.Partner', 'customer.Dependents', 'customer.tenure', 'phone.PhoneService', 'phone.MultipleLines', 'internet.InternetService', 'internet.OnlineSecurity', 'internet.OnlineBackup', 'internet.DeviceProtection', 'internet.TechSupport', 'internet.StreamingTV', 'internet.StreamingMovies', 'account.Contract', 'account.PaperlessBilling', 'account.PaymentMethod', 'account.Charges.Monthly', 'account.Charges.Total', 'Contas_Diarias']

Nomes das colunas DEPOIS da padronização:
['customerID', 'Churn', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'Monthly', 'Total', 'Contas_Diarias']

Colunas ['Churn', 'Partner', 'Depend

/tmp/ipython-input-15-3049600255.py:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dados_clientes[coluna] = dados_clientes[coluna].replace(mapa_sim_nao)


#📊 Carga e análise

In [16]:
import pandas as pd

pd.set_option('display.float_format', lambda x: '%.2f' % x)


print("--- Análise Descritiva das Variáveis Numéricas ---")
display(dados_clientes.describe().T)


print("\n\n--- Análise Descritiva das Variáveis Categóricas ---")

display(dados_clientes.describe(include=['object']).T)

--- Análise Descritiva das Variáveis Numéricas ---


,count,mean,std,min,25%,50%,75%,max
Churn,7032.00,0.27,0.44,0.00,0.00,0.00,1.00,1.00
SeniorCitizen,7032.00,0.16,0.37,0.00,0.00,0.00,0.00,1.00
Partner,7032.00,0.48,0.50,0.00,0.00,0.00,1.00,1.00
Dependents,7032.00,0.30,0.46,0.00,0.00,0.00,1.00,1.00
tenure,7032.00,32.42,24.55,1.00,9.00,29.00,55.00,72.00
PhoneService,7032.00,0.90,0.30,0.00,1.00,1.00,1.00,1.00
PaperlessBilling,7032.00,0.59,0.49,0.00,0.00,1.00,1.00,1.00
Monthly,7032.00,64.80,30.09,18.25,35.59,70.35,89.86,118.75
Total,7032.00,2283.30,2266.77,18.80,401.45,1397.47,3794.74,8684.80
Contas_Diarias,7032.00,2.16,1.00,0.61,1.19,2.34,3.00,3.96




--- Análise Descritiva das Variáveis Categóricas ---


,count,unique,top,freq
customerID,7032,7032,9995-HOTOH,1
gender,7032,2,Male,3549
MultipleLines,7032,3,No,3385
InternetService,7032,3,Fiber optic,3096
OnlineSecurity,7032,3,No,3497
OnlineBackup,7032,3,No,3087
DeviceProtection,7032,3,No,3094
TechSupport,7032,3,No,3472
StreamingTV,7032,3,No,2809
StreamingMovies,7032,3,No,2781


#📄Relatorio Final